In [29]:
!pip install astropy[recommended] --upgrade
!pip install ipympl
!pip install graphviz pandas

%matplotlib ipympl
from glob import glob
from re import sub
from astropy.io import fits
from astropy.units.cgs import C
import numpy as np
import matplotlib.pyplot as pl
from astropy.visualization import astropy_mpl_style
import statistics
from scipy.signal import argrelextrema
from astropy.modeling import models, fitting
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, HBox
import ipywidgets as widgets

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [30]:
try:
  tried = glob("Data/Input/*")[1]
except:
  tried = "Error"
try:
   ctried = glob("Data/Calibration/Default/*")[0]
except:
  ctried = "Error"

def data(filepath = tried, dflip = False):
  image_data = []
  print(f"Vorbereitung der Bilddatei: {filepath}")
  if filepath.endswith(".fits") is True:
    if filepath.startswith("spectrum_calibration/"):
        filepath = filepath.replace("spectrum_calibration/","")
    image_data = fits.getdata(f"{filepath}", ext=0)
    print("Fits-Format-Bild erkannt")
    image_array = np.array(image_data)
    prepped_array = image_array
    if dflip == True:
      prepped_array = np.fliplr(np.flipud(prepped_array))
      print("Spiegelung der Werte für Auswertung")
    else:
      print("Keine Spiegelung der Werte für Auswertung")
    print(f"Dimensionen des Bildes:{image_array.shape}")
    return prepped_array
  else:
    print("Invalides Bildformat")

def prep(prepped_array):
  ymaxpos = np.argmax(prepped_array, axis = 0)
  xmaxpos = np.arange(prepped_array.shape[1])
  ymaxvals = []
  for val in range(len(ymaxpos)):
    ymaxvals.append(prepped_array[ymaxpos[val],xmaxpos[val]])
  return ymaxpos, xmaxpos, ymaxvals

def plotreset(mode: bool = False):
    pl.ion()
    pl.axvline.linewidth = 0.00002
    pl.plot.linewidth = 0.005
    if mode == True:
        pl.yscale('log')
    else:
        pl.yscale("linear")
    pl.style.use(astropy_mpl_style)

def calibfile():
  fcalib = open("Data/Calibration/Calibrations.txt","r")
  calib = fcalib.readlines()
  fcalib.close()
  calibs = {}
  for n in range(len(calib)):
    newcalib = []
    for e in calib[n].split(": ")[1].split(","):
        try:
            newcalib.append(float(e))
        except:
            pass
    calibs[str(calib[n].split(": ")[0])] = newcalib
  return calibs, calib

In [31]:
def spectrum(file=tried,calibration="Temp",polydegree=3,flip=True,mode="automedian",average=True,part=0.9,reach=10,setsize=10,linepolydegree=3,safetyoversize=1.2,show="None",logarithmic=False
             #,normed=False
            ):
  prepped_array = data(filepath=file,dflip=flip)
  poss = prep(prepped_array)
  xmaxpos = poss[1]
  specvals = specvalues(image=prepped_array,Smode=mode,Saverage=average,
                        Spart=part,Sreach=reach,Ssetsize=setsize,Ssafetyoversize=safetyoversize,Slinepolydegree=linepolydegree)
  if calibration != "None":
      scaledpos = scale(xmaxpos,calibration,sshow=show,scpolydegree=polydegree)
      xmaxpos = scaledpos[0]

  #if normed == True:
   #   normedspecvals = []
    #  polynomial = np.poly1d(np.polyfit(np.array(xmaxpos), np.array(specvals), polydegree))
     # for i in range(len(xmaxpos)):
      #   normedspecvals.append(specvals[i] / polynomial(xmaxpos[i]))
  
  pl.figure(num="Spektrum",clear=True)    
  plotreset(logarithmic)

  #if normed == True:
   #   pl.plot(xmaxpos, polynomial(xmaxpos), label = "polynorm")
      
  if average == True:
    pl.ylabel("durchnittliche Datenwerte")
  else:
    pl.ylabel("addierte Datenwerte")
  if calibration == "None":
    pl.xlabel("X-Position")
    pl.plot(xmaxpos, specvals, label = "Spektrum")
  else:
    pl.xlabel("Wellenlänge in nm")
    pl.plot(xmaxpos, specvals, label = "Spektrum")
    if show == "Markers" or show == "Both":
        for e in scaledpos[1]:
            pl.axvline(x = e, color = "red")
  pl.show("Spektrum")
  print("Spektrum darstellen")

  #if normed == True:
  #  pl.figure(num="Normiertes Spektrum",clear=True)
   # plotreset(logarithmic)
    #pl. ylabel("Normierte Werte")
  if calibration == "None":
      pl.xlabel("X-Position")
  else: 
      pl.xlabel("Wellenlänge in nm")
  #pl.plot(xmaxpos, normedspecvals, label = "Normiertes Spektrum")
  #pl.show("Normiertes Spektrum")

In [32]:
def specvalues(image,Smode="automedian",Saverage=True,Spart=0.8,Sreach=10,Ssetsize=10,Slinepolydegree=3,Ssafetyoversize=1.2):
  poss = prep(image)
  ymaxpos = poss[0]
  xmaxpos = poss[1]
  ymaxvals = poss[2]
  specvals = []
  if Smode == "automedian":
    yvalues = []
    for x in range(len(xmaxpos)):
      ymedian = []
      for y in range(image.shape[0]):
          ymedian.append(image[(y, xmaxpos[x])])
      yvalues = ymedian
      ymedian = statistics.median(ymedian)
      averaged = 0
      delta = image[ymaxpos[x], x] - ymedian
      low = ymaxvals[x] - (Spart * delta)
      values = []
      for extract in yvalues:
          if extract >= low and extract <= ymaxvals[x]:
              values.append(int(extract))
      averaged = sum(values)
      if Saverage == True and len(values) > 0:
          specvals.append(averaged/len(values))
      else:
          specvals.append(averaged)
  elif Smode == "moundfind":
      for x in range(len(xmaxpos)):
          ymedian = []
          ymax = ymaxvals[x]
          for y in range(image.shape[0]):
            ymedian.append(int(image[(y, xmaxpos[x])]))
          yvalues = ymedian
          ymaxes = ymedian
          ymaxes.sort(reverse=True)
          counter = 0
          ymedian = statistics.median(ymedian)
          satisfied = False
          while satisfied == False:
              mound = []
              collecting = False
              averaged = 0
              delta = ymax - ymedian
              low = ymax - (Spart * delta)
              for extract in yvalues:
                  if extract >= low and extract <= ymaxvals[x]:
                      if collecting == False and len(mound) == 0:
                          collecting = True
                          mound.append(int(extract))
                      else:
                          mound.append(int(extract))
                  else:
                      collecting = False
              if len(mound) >= 2 * Sreach:
                  satisfied = True
              else:
                  counter += 1
                  ymax = ymaxes[counter]
          averaged = sum(mound)
          if Saverage == True and len(mound) > 0:
              specvals.append(averaged/len(mound))
          else:
              specvals.append(averaged)
  elif Smode == "aroundline":
      linepos = createimlinearray(ymaxpos, xmaxpos,setsize=Ssetsize,linepolydegree=Slinepolydegree)
      for x in range(len(xmaxpos)):
          values = []
          ypointpos = linepos[xmaxpos[x]]
          for y in range(int(ypointpos-Sreach), int(ypointpos+Sreach+1)):
              if y <= image.shape[0] and y >= 0:
                  values.append(int(image[y, xmaxpos[x]]))
          averaged = sum(values)
          if Saverage == True and len(values) > 0:
              specvals.append(averaged/len(values))
          else:
              specvals.append(averaged)
  elif Smode == "combined":
      linepos = createimlinearray(ymaxpos, xmaxpos,setsize=Ssetsize,linepolydegree=Slinepolydegree)
      for x in range(len(xmaxpos)):
          values = []
          ypointpos = linepos[xmaxpos[x]]
          for y in range(int(ypointpos-Sreach), int(ypointpos+Sreach+1)):
              if y <= image.shape[0] and y >= 0:
                  values.append(int(image[y, xmaxpos[x]]))
          for e in values:
              if e < (1 - Spart) * int(image[ypointpos,xmaxpos[x]]) or e > Ssafetyoversize * int(image[ypointpos,xmaxpos[x]]):
                  values.remove(e)
          averaged = sum(values)
          if Saverage == True and len(values) > 0:
                  specvals.append(averaged/len(values))
          else:
              specvals.append(averaged)
  elif Smode == "autoaverage":
    for x in range(len(xmaxpos)):
      averaged = 0
      yaverage = 0
      yvalues = []
      for y in range(image.shape[0]):
        yaverage += image[(y,xmaxpos[x])]
        yvalues.append(image[(y,xmaxpos[x])])
      yaverage = yaverage / image.shape[0]
      averaged = 0
      values = []
      for extract in yvalues:
        if extract > yaverage:
          averaged += int(extract)
          values.append(int(extract))
      if Saverage == True and len(values) > 0:
          specvals.append(averaged/len(values))
      else:
        specvals.append(averaged)
  elif Smode == "ranged":
    for x in range(len(xmaxpos)):
      averaged = 0
      for y in range(-Sreach,Sreach+1):
        try:
            averaged += image[(y+ymaxpos[x], xmaxpos[x])]
        except:
            pass
      if Saverage == True:
        specvals.append(averaged/(Sreach*2+1))
      elif Saverage == False:
        specvals.append(averaged)
  specvals = np.array(specvals)
  print(f"Spektrum erstellt im Modus: {Smode}")
  return specvals

In [33]:
def calibrate(file=ctried,flip=True,wl="WL_Q",mode="automedian",average=True,region=80.0,reach=10,
              width=0.7,part=0.75,setsize=10,linepolydegree=3,safetyoversize=1.2,minimum=100,search=True,wanted=9,logarithmic=False):
  pl.figure(num="Kalibration",clear=True)
  plotreset(logarithmic)
  prepped_array = data(filepath=file,dflip=flip)
  poss = prep(prepped_array)
  Cxmaxpos = poss[1]
  Cspecvals = specvalues(image=prepped_array,Smode=mode,Saverage=average,
                        Spart=part,Sreach=reach,Ssetsize=setsize,Ssafetyoversize=safetyoversize,Slinepolydegree=linepolydegree)
  maximapos = argrelextrema(Cspecvals, np.greater, order = int(region))
  gaussmaxima = []
  maximapos = maximapos[0]
  calibs = calibfile()
  calib = calibs[1]
  calibs = calibs[0]
  print("Suche nach spezifizierten Maximalstellen")
  if search == True:  
      xmaximapos = range(1000)
      while len(xmaximapos) > len(calibs[wl]) and len(xmaximapos) > wanted:
          xmaximapos = []
          for ind in maximapos:
              if Cspecvals[ind] > minimum:
                  xmaximapos.append(Cxmaxpos[ind])
              minimum += 1
      print(f"Suche beendet bei Minimum: {minimum}")
  else:
      xmaximapos = []
      for ind in maximapos:
              if Cspecvals[ind] > minimum:
                  xmaximapos.append(Cxmaxpos[ind])
  maximaspec = []
  for x in xmaximapos:
    maximaspec.append(Cspecvals[x])
  #pl.plot(xmaximapos, maximaspec, label = 'local maxima', marker = 'D')
  for maximum in xmaximapos:
    xtempregion = []
    ytempregion = []
    for e in range(int(-width*region), int(width*region)):
      if (maximum + e) > 0 and ((maximum + e) < (len(Cxmaxpos) - 1)):
        xtempregion.append(Cxmaxpos[maximum + e])
        ytempregion.append(Cspecvals[maximum + e])
    xgaussregion = range(len(xtempregion))
    g_init = models.Gaussian1D(amplitude=1., mean=0, stddev=1.)
    fit_g = fitting.LevMarLSQFitter()
    g = fit_g(g_init, xgaussregion, ytempregion)
    #print(xtempregion[argrelextrema(g(xgaussregion)
                                          #  ,np.greater,order=int(region))[0][0]])
    try:
        pl.axvline(x=xtempregion[argrelextrema(g(xgaussregion)
                                            ,np.greater,order=int(region))[0][0]],color="red")
        gaussmaxima.append(float(xtempregion[argrelextrema(g(xgaussregion)
                                                           , np.greater, order = int(region))[0][0]]))
    except:
        pass
  #pl.plot(xtempregion, g(xgaussregion), label='Gaussian') 
  for indexer, element in enumerate(calib):
    if "Temp:" in element:
      tempstr = "Temp: "
      for index, e in enumerate(calibs[wl]):
        if index < len(gaussmaxima):
            tempstr += f"{gaussmaxima[index]}, " 
            if index == len(calibs[wl])-1:
                tempstr += f"{e}"
            else:
                tempstr += f"{e}, "
      tempstr += "\n"
      calib[indexer] = tempstr
  print(f"Spezifizierte Maximalstellen gefunden: {gaussmaxima}")
  print("Wellenlängenzuweisungen:")
  tempstr = tempstr.split(": ")[1].split("\n")[0].split(", ")
  for w in range(0,len(tempstr),2):
      try:
          print(f"X-Stelle: {tempstr[w]} -> {tempstr[w+1]} nm")
      except:
          pass
  pl.xlabel("X-Position")
  if average == True:
    pl.ylabel("durchnittliche Datenwerte")
  else:
    pl.ylabel("addierte Datenwerte")
  pl.plot(Cxmaxpos, Cspecvals, label = "Spektrum",color = "black")
  print("Darstellen der gefundenen Maximalstellen")
  pl.show("Kalibration")
  temp = open("Data/Calibration/Calibrations.txt", "w")
  temp.write("")
  temp.close()
  temp = open("Data/Calibration/Calibrations.txt","a")
  for line in calib:
    temp.write(line)
  temp.close()
  print("Kalibrierung temporär gespeichert")

In [34]:
def scale(koords, calibration, sshow = "None",scpolydegree=3):
  calibs = calibfile()
  ccalib = calibs[0][str(calibration)]

  wavepos = []
  pixpos = []
  for e in range(len(ccalib)):
    if e % 2 == 0:
        pixpos.append(ccalib[e])
    else:
        wavepos.append(ccalib[e])

  polynomial = np.poly1d(np.polyfit(np.array(pixpos), np.array(wavepos), scpolydegree))
  
  scaledpos = polynomial(koords)

  markers = []
  if sshow == "Markers" or sshow == "Both":
    for n in range(len(ccalib)):
      if n % 2 != 0:
        markers.append(ccalib[n])

  if sshow == "Scaling" or sshow == "Both":
      pl.figure(num="Wellenlängenskalierung",clear=True)
      plotreset()
      pl.ylabel("Wellenlänge in nm")
      pl.xlabel("X-Position")
      pl.scatter(pixpos,wavepos)
      pl.plot(koords, polynomial(koords))
      pl.show("Wellenlängenskalierung")
    
  print(f"Erste und Letzte Wellenlänge der kalibrierten Skala:{float(scaledpos[0]),float(scaledpos[-1])}")
  return [scaledpos,markers]

In [35]:
def showimg(file = tried, flip = True,show=False,setsize=10,linepolydegree=3):
    pl.figure(num="Anzeige",clear=True)
    plotreset()
    image_array = data(file, dflip = flip)
    prepped = prep(image_array)
    print("Bild anzeigen")
    pl.imshow(image_array)
    if show == True:
        pl.plot(prepped[1],createimlinearray(prepped[0],prepped[1], setsize,linepolydegree))
    pl.show("Anzeige")

In [36]:
def createimlinearray(ypos, xpos, setsize,linepolydegree=3):
    linearray = []
    print(linepolydegree)
    yposmedian = statistics.median(ypos)
    setmeds = []
    xsetpos = []
    if setsize != 1:
        for setnum in range(int(len(xpos)/setsize)):
            set = []
            for e in range(setsize):
                if e + setsize * setnum <= len(xpos):
                    set.append(ypos[xpos[e + setsize * setnum]])
                elif len(set) > 0:
                    set.append(set[-1])
                else:
                    set.append(yposmedian)
            setmeds.append(statistics.median(set))
            xsetpos.append(int(setnum*setsize+0.5*setsize))
        polynomial = np.poly1d(np.polyfit(np.array(xsetpos), np.array(setmeds), linepolydegree))
    else:
        polynomial = np.poly1d(np.polyfit(np.array(xpos), np.array(ypos), linepolydegree))
    for i in xpos:
        if int(polynomial(i)) > 0 and int(polynomial(i)) < max(ypos):
            linearray.append(int(polynomial(i)))
        else:
            linearray.append(int(yposmedian))
    return linearray
    

In [37]:
print("\nBildanzeige:")
showimg_widget = interactive(showimg,{'manual':True, 'manual_name': 'Anzeigen'},
                            setsize=widgets.IntSlider(min=1,max=200,step=1,value=10),
                            linepolydegree=widgets.IntSlider(min=1,max=20,value=3))
display(showimg_widget)


Bildanzeige:


interactive(children=(Text(value='Data/Input/MED_sonne240212 4.fits', continuous_update=False, description='fi…

In [38]:
print("\nKalibration:")
calibration_widget = interactive(calibrate,{'manual': True, 'manual_name': 'Ausführen'},
                                mode = ['moundfind','automedian', 'autoaverage','ranged','aroundline','combined'],
                                region=widgets.IntSlider(min=1,max=200,step=1,value=40),
                                minimum=widgets.FloatSlider(min=1,max=10000,step=1,value=200),
                                width=widgets.FloatSlider(min=0.01,max=2,step=0.01,value=0.7),
                                wanted=widgets.IntSlider(min=1,max=30,step=1,value=9),
                                part=widgets.FloatSlider(min=0.01,max=1,step=0.01,value=0.75),
                                setsize=widgets.IntSlider(min=1,max=200,step=1,value=10),
                                safetyoversize=widgets.FloatSlider(min=0.5,max=5.0,step=0.05,value=2.0),
                                linepolydegree=widgets.IntSlider(min=1,max=20,value=3)) 
display(calibration_widget)


Kalibration:


interactive(children=(Text(value='Data/Calibration/Default/calib.fits', continuous_update=False, description='…

In [39]:
print("\nSpektrum:")
spectrum_widget = interactive(spectrum,{'manual': True, 'manual_name': 'Ausführen'}, 
                              mode = ['moundfind','automedian', 'autoaverage','ranged','aroundline','combined'],
                              show = ["None","Markers","Scaling","Both"],
                              part=widgets.FloatSlider(min=0.01,max=1,step=0.01,value=0.9),
                             reach=widgets.IntSlider(min=1,max=100,step=1,value=10),
                             setsize=widgets.IntSlider(min=1,max=200,step=1,value=10),
                             safetyoversize=widgets.FloatSlider(min=0.5,max=5.0,step=0.05,value=2.0),
                             polydegree=widgets.IntSlider(min=1,max=10,step=1,value=3),
                             linepolydegree=widgets.IntSlider(min=1,max=20,value=3))
display(spectrum_widget)


Spektrum:


interactive(children=(Text(value='Data/Input/MED_sonne240212 4.fits', continuous_update=False, description='fi…